# Análisis de algoritmos

In [35]:
def sub_array_sum_aux(arr, m):
    if m == 0:
        return arr[0]
    else:
        return max(arr[m], arr[m] + sub_array_sum_aux(arr, m-1))

def sub_array_sum(arr, n):
    max_sum = -float("inf")
    for i in range(0, n):
        max_sum = max(max_sum, sub_array_sum_aux(arr, i))
    return max_sum

La función auxiliar es una función recursiva cuyo tiempo es de la forma $t(m)=t(m-1) + C\in \Theta(m).$ La función principal consta de un simple bucle que llama a la función auxiliar: $t(n) = C + \sum_i \Theta(i)\in \Theta(n^2).$

## Incidencias

In [1]:
def busqueda_recursiva(arr, x, left, right):
    if right == left:
        return arr[right] == x
    else:
        mid = (left + right) // 2
        encontrado_l = busqueda_recursiva(arr, x, left, mid)
        if encontrado_l:
            return True
        encontrado_d = busqueda_recursiva(arr, x, mid + 1, right)
        return encontrado_d

En el mejor caso, nunca va a explorar el lado derecho: $t(n)=t(n/2)+C$. Usando el teorema maestro, $t(n)\in\Theta(\log n)$.

En el peor caso, explora las dos mitades en toda llamada: $t(n)=2t(n/2)+C$. Usando el teorema maestro, $t(n)\in\Theta( n)$.

No hay un orden exacto porque los órdendes del peor y mejor caso no coinciden.

La búsqueda tradicional es $\Theta(n)$ y $\Theta(1)$ en el peor y mejor caso respectivamente. Por tanto, esta versión recursiva es peor ya que, el tiempo en el mejor caso es peor ($\Theta(1)$ vs $\Theta(\log n)$).

# DyV

In [1]:
def f(arr, i, m):
    """
    Ejemplo de f para que compile
    """
    s = arr[i:i+m]
    return len(s) == m and s == ''.join(sorted(s))

def frontera(arr, left, mid, right, m):
    cont = 0
    # Check all substrings of length m that might cross the midpoint
    for i in range(mid - m + 2, mid + 1):
        if i >= left and i + m <= right + 1:
            if f(arr, i, m):
                cont += 1
    return cont

def dyv(arr, left, right, m):
    if right - left + 1 < m:
        return 0
    else:
        mid = (left + right) // 2
        cont_l = dyv(arr, left, mid, m)
        cont_r = dyv(arr, mid + 1, right, m)
        cont_m = frontera(arr, left, mid, right, m)
        return cont_l + cont_r + cont_m



In [6]:
# Test Case 1: All increasing substrings
assert dyv("abcdef", 0, 5, 3) == 4

# Test Case 2: Some sorted substrings
assert dyv("acbd", 0, 3, 2) == 2

# Test Case 3: No sorted substrings
assert dyv("dcba", 0, 3, 2) == 0

# Test Case 4: String shorter than m
assert dyv("abc", 0, 2, 5) == 0

# Test Case 5: Duplicates in sorted substring
assert dyv("aabbcc", 0, 5, 2) == 5

# Test Case 6: Entire string is sorted and matches m
assert dyv("abc", 0, 2, 3) == 1

# Test Case 7: Only one valid sorted substring
assert dyv("zxyab", 0, 4, 2) == 2  # only "ab" is sorted


El tiempo es $t(n,m)=2t(n/2,m) + f(m).$ Como se supone $m$ constante, entonces $t(n)=2t(n/2)+\Theta(1) \in \Theta(n)$ según el teorema maestro.

## DyV incidencias:

In [15]:
def f(arr, i, m):
    s = arr[i:i+m]
    return len(s) == m and s == ''.join(sorted(s))

def frontera(arr, left, mid, right, m):
    # Check all substrings of length m that might cross the midpoint
    for i in range(mid - m + 2, mid + 1):
        if i >= left and i + m <= right + 1:
            if f(arr, i, m):
                return True
    return False

def dyv(arr, left, right, m):
    if right - left + 1 < m:
        return False
    else:
        mid = (left + right) // 2
        cont_l = dyv(arr, left, mid, m)
        cont_r = dyv(arr, mid + 1, right, m)
        cont_m = frontera(arr, left, mid, right, m)
        return cont_l or cont_r or cont_m

In [16]:
assert dyv("abcdef", 0, 5, 3) == True  # "abc" es una subcadena ordenada de longitud 3
assert dyv("abdecf", 0, 5, 2) == True  # "ab", "cf", etc.
assert dyv("bacdef", 0, 5, 4) == True  # "cdef" es válida
assert dyv("azbycxdwev", 0, 9, 2) == True  # longitud 1 siempre es válida (1 letra está ordenada)


In [17]:
assert dyv("zyxwv", 0, 4, 2) == False  # Todo va en orden inverso
assert dyv("abc", 0, 2, 4) == False  # Subcadena de longitud 4 no cabe
assert dyv("a", 0, 0, 2) == False  # Longitud insuficiente
assert dyv("edcba", 0, 4, 3) == False  # Ninguna subcadena ordenada de 3


El tiempo es $t(n,m)=2t(n/2,m) + f(m)$ y se hacen todas las recursiones porque no existe ninguna subcadena que cumpla la condición. Como se supone $m$ constante, entonces $t(n)=2t(n/2)+\Theta(1) \in \Theta(n)$ según el teorema maestro.

# Voraces

La decisión voraz: escoger la estación alcanzable más alejada.

In [29]:
def voraz(D, C, E):
    E.append(D)
    num_stops = 0
    current_pos = 0
    i = 0
    n = len(E)
    S = []

    while current_pos + C < D:

        next_stop = i
        while next_stop < n and E[next_stop] <= current_pos + C:
            next_stop += 1

        if i == next_stop:
            return "No sol" 

        num_stops += 1
        current_pos = E[next_stop - 1]
        S.append(current_pos)
        i = next_stop

    return S


In [31]:
E = [100, 200, 300, 400, 500, 700]
D = 800
C = 200
voraz(D, C, E)

[200, 400, 500, 700]

# PD

Este problema es una versión de fibonacci pero desplazada. La ecuación de recurrencia es $dp(i) = dp(i-1) + dp(i-2)$. Los casos base son: $dp(1) = 1$ y $dp(2) = 2$.

In [30]:
def programacion_dinamica(n):
    dp = [0]*(n+1)
    for i in range(1, n + 1):
        if i == 1:
            dp[i] = 1
        elif i == 2:
            dp[i] = 2
        else:
            dp[i] = dp[i-1] + dp[i-2]
    return dp[n]

In [31]:
programacion_dinamica(27)

317811

## Incidencias

La ecuación de recurrencia es $dp(i) = dp(i-1) + dp(i-2) + dp(i-3)$. Los casos base son: $dp(1) = 1$, $dp(2) = 2$, $dp(3)=4$.

In [32]:
def programacion_dinamica(n):
    dp = [-1]*(n+1)
    for i in range(1, n + 1):
        if i == 1:
            dp[i] = 1
        elif i == 2:
            dp[i] = 2
        elif i == 3:
            dp[i] = 4
        else:
            dp[i] = dp[i-1] + dp[i-2] + dp[i-3]
    return dp[n]

In [34]:
programacion_dinamica(4)

7

# Backtracking

En este caso $s=(x_1,\dots,x_n)$ donde cada $x_i\in\{1,\dots,n\}$ va a ser un nodo del grafo. Estamos ante un árbol permutacional. La principal dificultad del algoritmo recae en el *generar*, *valor* y *retroceder*. En *generar*, hay que tener en cuenta 3 casos: si es el primer nivel, si no es el primer nivel pero es el primer hermano y si no es el primer nivel y tampoco el primer hermano. En el *valor* hay que sumar la arista del último nodo con el primero. En *retroceder* hay que tener en cuenta que `coste_actual` solo se modifica cuando `nivel > 1`.

In [75]:
def solucion(G, nivel, s, visitados):
    return visitados[s[nivel]] == 1 and len(G) == nivel

def criterio(G, nivel, s, visitados):
    return nivel < len(G) and visitados[s[nivel]] == 1

def valor(G, s, coste_actual, nivel):
    return coste_actual + G[s[nivel]][s[1]]['weight']

def backtracking(G):
    s = [0]*(len(G) + 1)
    visitados = [0]*(len(G) + 1)
    coste_actual = 0
    voa = float("inf")
    soa = None
    nivel = 1

    while nivel != 0:
        # genero un hermano
        s[nivel] += 1
        if nivel == 1:
            visitados[s[nivel]] += 1
        else:
            if s[nivel] == 1:
                coste_actual += G[s[nivel - 1]][s[nivel]]['weight']
            else:
                visitados[s[nivel]-1] -= 1
                coste_actual += G[s[nivel - 1]][s[nivel]]['weight']
                coste_actual -= G[s[nivel - 1]][s[nivel]-1]['weight']
            visitados[s[nivel]] += 1

        if solucion(G, nivel, s, visitados) and valor(G, s, coste_actual, nivel) < voa:
            voa = valor(G, s, coste_actual, nivel)
            soa = s.copy()

        if criterio(G, nivel, s, visitados) and coste_actual < voa:
            nivel += 1
        else:
        # en caso contario, retrocedo hasta que encuentre un nodo con más hermanos
        # por explorar
            while nivel > 0 and (not s[nivel] < len(G)):
                if nivel > 1:
                    coste_actual -= G[s[nivel - 1]][s[nivel]]['weight']
                visitados[s[nivel] ] -= 1
                s[nivel] = 0
                nivel -= 1
    return soa, voa

In [76]:
import networkx as nx

G = nx.Graph()

# Add 4 nodes with some random distances (edges with weights)
G.add_edge(1, 2, weight=10)
G.add_edge(2, 3, weight=20)
G.add_edge(3, 4, weight=30)
G.add_edge(4, 1, weight=40)
G.add_edge(1, 3, weight=50)
G.add_edge(2, 4, weight=60)
G.add_edge(1, 1, weight=0)
G.add_edge(2, 2, weight=0)
G.add_edge(3, 3, weight=0)
G.add_edge(4, 4, weight=0)

In [77]:
backtracking(G)

([0, 1, 2, 3, 4], 100)

El orden del algoritmo es $O(n!)$ ya que el tiempo por nodo es constante y el número de nodos es, en el peor de los casos, $O(n!)$ ya que estamos ante un arbol permutacional.

## Incidencias


Lo único que cambia es `valor`:

```python
def valor(G, s, coste_actual, nivel):
    return coste_actual #se elimina esto ->+ G[s[nivel]][s[1]]['weight']
```